In [5]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
url = "/Users/adishsundar/Desktop/MLDS/Amazon_reviews_plus_LLM.csv"
data = pd.read_csv(url)

In [8]:
# 1422 rows with null data, getting rid of them can improve the accuracy of our results
new_data = data
new_data.dropna(inplace=True)

In [9]:
new_data.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,llm
0,c23c3405-0e6f-44d5-b8bd-cf3ae7af7507,ed4114b9-858d-446b-b959-0c3ea0cda107,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1.0,Don't waste your money,1391040000,"01 30, 2014",False
1,43451c1d-99bc-4656-ad09-78bc3c2bb5b1,ed4114b9-858d-446b-b959-0c3ea0cda107,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3.0,OK Palette!,1397779200,"04 18, 2014",False
2,34307dbe-a180-4cbe-a928-0554828a66a8,ed4114b9-858d-446b-b959-0c3ea0cda107,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4.0,great quality,1378425600,"09 6, 2013",False
3,61400f04-a1b6-41ba-a2ee-297580fe5939,ed4114b9-858d-446b-b959-0c3ea0cda107,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2.0,Do not work on my face,1386460800,"12 8, 2013",False
4,df6d6c81-6817-46d9-8fed-03bc278c95b6,ed4114b9-858d-446b-b959-0c3ea0cda107,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3.0,It's okay.,1382140800,"10 19, 2013",False


In [10]:
df = new_data
df[['num1', 'num2']] = df['helpful'].str.replace('[', '').str.replace(']', '').str.split(',', expand=True).astype(int)

/var/folders/bn/cbv7ftpx4cx24wl0rz52v1z00000gn/T/ipykernel_8013/528155117.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[['num1', 'num2']] = df['helpful'].str.replace('[', '').str.replace(']', '').str.split(',', expand=True).astype(int)


In [11]:
count = 0
for index, row in new_data.iterrows():
    if (int(row['num1'])) <= ((int(row['num1']) + int(row['num2'])) * 0.3):
        count += 1

In [12]:
print(count/len(data))

0.6579826583034353


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [15]:
X_train, X_test, y_train, y_test = train_test_split(new_data['reviewText'], new_data['llm'], random_state=42)

# Create a TfidfVectorizer object to convert the text data into numerical features
vectorizer = TfidfVectorizer(stop_words='english')

# Fit the vectorizer on the training data
X_train_vec = vectorizer.fit_transform(X_train)

# Create a logistic regression classifier and fit it on the training data
clf = LogisticRegression(max_iter=10000)
clf.fit(X_train_vec, y_train)

# Use the classifier to make predictions on the test data
X_test_vec = vectorizer.transform(X_test)
y_pred = clf.predict(X_test_vec)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

report = classification_report(y_test, y_pred)
print('Classification Report:', report)

Accuracy: 0.985510965475039
Classification Report:               precision    recall  f1-score   support

       False       0.99      1.00      0.99     49279
        True       0.90      0.52      0.66      1380

    accuracy                           0.99     50659
   macro avg       0.95      0.76      0.83     50659
weighted avg       0.98      0.99      0.98     50659



In [17]:
# The average reviewes per reviewer overall 
count_per_reviewer = new_data.groupby('reviewerID')
newest = count_per_reviewer.count().sort_values(by='asin', ascending=False)
newest['asin'].mean()

8.132319300076253

In [20]:
# The average reviewes per reviewer where ChatGPT was used to generate a fake review
True_GPT = new_data[new_data['llm'] == True]
gpt = True_GPT.groupby('reviewerID').count().sort_values(by='asin', ascending=False)
gpt['asin'].mean()

2.157403808783521

In [18]:
count = 0 

count_2 = 0

count_3 = 0

count_gpt = 0

total_count = 0

for index, row in new_data.iterrows():
    if "nail" in row["reviewerName"].lower():
        count += 1
        
    if "nail" in row["reviewerName"].lower() and "nail" in row["reviewText"].lower():
        count_2 += 1
        
    if "nail" in row["reviewerName"].lower() and "nail" in row["reviewText"].lower() and "nail" in row["summary"].lower():
        count_3 += 1
        
    if "nail" in row["reviewerName"].lower() and row['llm'] == True:
        count_gpt += 1
        
    total_count += 1
   
print(count)
print(count_2)
print(count_3)
print(count_gpt)
print(total_count)

291
125
26
96
202633


In [19]:
# roughly 30-40% of our results show that product names are coorelated to fake names. (We tested with various products)
96 / 291

0.32989690721649484

In [21]:
# Percent of rows that are flagged True for ChatGPT, roughly 3%
len(True_GPT) / len(new_data)

0.027394353338301265

In [22]:
# average number of characters for reviews that were not guaranteed to be generated by ChatGPT

count = 0

for index, row in new_data.iterrows():
    if row['llm'] == False:
        count += len(row['reviewText'])
        
print(count/len(new_data))

459.83059027897724


In [23]:
# average number of characters for reviews that were generated by ChatGPT

count = 0

for index, row in True_GPT.iterrows():
    count += len(row['reviewText'])
        
print(count/len(True_GPT))

234.43901999639704


In [35]:
# Looking at the amount of reviews from the Chat GPT generated reviews, we can look at the product ID's with the 
# highest number of reviews.
# Olay Regenerist Night Recovery Moist Treatment is the product with the most fradulent reviews
# 22/132 = 16%

True_GPT.groupby('asin').count().sort_values(by='reviewerID', ascending = False).head()

,reviewerID,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,llm,num1,num2
asin,,,,,,,,,,,
b1d65a8c-bcdd-4b53-aac8-43366bf28424,22,22,22,22,22,22,22,22,22,22,22
bcdee70e-f091-4a7d-a177-0ac444056258,21,21,21,21,21,21,21,21,21,21,21
44ca3f0f-e5d3-4afb-975a-60513ac7b4c9,20,20,20,20,20,20,20,20,20,20,20
d9ba2708-c770-4b78-bd47-8044b5de830c,19,19,19,19,19,19,19,19,19,19,19
5bbb4054-c2be-4dec-9d5b-cd8e45be0962,19,19,19,19,19,19,19,19,19,19,19
